<a href="https://colab.research.google.com/github/trivedidharmik/cs3735/blob/main/Trivedi_Dharmik_Asmt3Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from google.colab import drive
from joblib import Parallel, delayed
# Store results
results = []

Load Breast Cancer data

In [ ]:
# Load Breast Cancer data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data"
column_names = ["id", "diagnosis"] + [f"feature_{i}" for i in range(1, 31)]
data = pd.read_csv(url, header=None, names=column_names)
X = data.drop(["id", "diagnosis"], axis=1)
y = data["diagnosis"].map({"M": 1, "B": 0})

Preprocess the data and set the hyperparameters

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

structures = [
    (64, 64),
    (128, 128),
    (64, 64, 64),
    (128, 128, 128)
]
activations = ['logistic', 'tanh', 'relu']
solvers = ['adam', 'sgd']

Run the experiments and return results dataframe

In [ ]:
# Experiments
for hidden_layer_sizes in structures:
    for activation in activations:
        for solver in solvers:
            mlp = MLPClassifier(
                hidden_layer_sizes=hidden_layer_sizes,
                activation=activation,
                solver=solver,
                random_state=42,
                max_iter=2000,
                tol=1e-4,
                early_stopping=False
            )
            mlp.fit(X_train_scaled, y_train)
            y_pred = mlp.predict(X_test_scaled)
            acc = accuracy_score(y_test, y_pred)
            results.append({
                'Structure': f"{len(hidden_layer_sizes)} layers {hidden_layer_sizes}",
                'Activation': activation,
                'Optimizer': solver,
                'Accuracy': f"{acc:.4f}"
            })
results_df = pd.DataFrame(results)
print("\n (Breast Cancer Data) Results Table:")
print(results_df.to_markdown(index=False))


 (Breast Cancer Data) Results Table:
| Structure                | Activation   | Optimizer   |   Accuracy |
|:-------------------------|:-------------|:------------|-----------:|
| 2 layers (64, 64)        | logistic     | adam        |     0.9825 |
| 2 layers (64, 64)        | logistic     | sgd         |     0.9825 |
| 2 layers (64, 64)        | tanh         | adam        |     0.9737 |
| 2 layers (64, 64)        | tanh         | sgd         |     0.9912 |
| 2 layers (64, 64)        | relu         | adam        |     0.9737 |
| 2 layers (64, 64)        | relu         | sgd         |     0.9649 |
| 2 layers (128, 128)      | logistic     | adam        |     0.9825 |
| 2 layers (128, 128)      | logistic     | sgd         |     0.9737 |
| 2 layers (128, 128)      | tanh         | adam        |     0.9649 |
| 2 layers (128, 128)      | tanh         | sgd         |     0.9825 |
| 2 layers (128, 128)      | relu         | adam        |     0.9737 |
| 2 layers (128, 128)      | relu      

Mount drive and load data for Digit Recognizer

In [ ]:
drive.mount("/content/drive", force_remount=True)


# Load and preprocess data once (outside the loop)
train_file = '/content/drive/My Drive/datasets/train.csv'
train_df = pd.read_csv(train_file)
X = train_df.drop(columns='label').values / 255.0
y = train_df['label'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a training function
def train_digit_model(hidden_layer_sizes, activation, solver):
    mlp = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        solver=solver,
        early_stopping=True,
        validation_fraction=0.1,
        max_iter=500,
        tol=1e-3,
        random_state=42
    )
    mlp.fit(X_train, y_train)
    acc = accuracy_score(y_val, mlp.predict(X_val))
    return {
        'Structure': f"{len(hidden_layer_sizes)} layers {hidden_layer_sizes}",
        'Activation': activation,
        'Optimizer': solver,
        'Accuracy': f"{acc:.4f}"
    }

# Run experiments in parallel
digit_results = Parallel(n_jobs=4)(
    delayed(train_digit_model)(hidden, act, solv)
    for hidden in structures
    for act in activations
    for solv in solvers
)

results_df = pd.DataFrame(digit_results)
print("\n (Breast Cancer Data) Results Table:")
print(results_df.to_markdown(index=False))

Mounted at /content/drive

 (Breast Cancer Data) Results Table:
| Structure                | Activation   | Optimizer   |   Accuracy |
|:-------------------------|:-------------|:------------|-----------:|
| 2 layers (64, 64)        | logistic     | adam        |     0.9637 |
| 2 layers (64, 64)        | logistic     | sgd         |     0.3969 |
| 2 layers (64, 64)        | tanh         | adam        |     0.9677 |
| 2 layers (64, 64)        | tanh         | sgd         |     0.9248 |
| 2 layers (64, 64)        | relu         | adam        |     0.9661 |
| 2 layers (64, 64)        | relu         | sgd         |     0.9465 |
| 2 layers (128, 128)      | logistic     | adam        |     0.9706 |
| 2 layers (128, 128)      | logistic     | sgd         |     0.8881 |
| 2 layers (128, 128)      | tanh         | adam        |     0.9715 |
| 2 layers (128, 128)      | tanh         | sgd         |     0.9461 |
| 2 layers (128, 128)      | relu         | adam        |     0.9714 |
| 2 layers (1